# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
print(azureml.core.VERSION)

1.21.0


In [ ]:
!pip install --upgrade --upgrade-strategy eager azureml-sdk[automl,widgets]

In [2]:

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn

import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-capstone'

experiment=Experiment(ws, experiment_name)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_DS3_V2"
compute_name="compute-cluster"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4




### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    print(compute_target.get_status().serialize())

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

<p>For automl conofiguration I chose:</p>
<p>Task as 'Classification' since the problem statement involves binary classfication (Malignant or Benign tumour)</p>
<p>The primary metric is accuracy as accuracy is a good parameter to learn about the model's performance</p>
<p>The training data is X which contains all the necessary features to identify the type of cancer</p>
<p>The test column is the target column :'Diagnosis' which tells about the type of tumour the patient has</p>


In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
#run = Run.get_context()
#ws = run.experiment.workspace
dataset = Dataset.get_by_name(ws, name='cancer')
dataset.to_pandas_dataframe()
df=dataset.to_pandas_dataframe()
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
df['diagnosis']= le.fit_transform(df['diagnosis']) 
df.head()

After using labelencoder on dignosis column the letter M which idicated malignant tumour is encoded with number 1 and benign tumour with 0.

In [7]:
#Splitting the dataset into X and y,X contains the first 10 columns which is used for training.
#y is the target column,which the model will predict
X=df.iloc[ :,1:10]
y=df.iloc[:,1]

In [ ]:
X

In [ ]:
y

In [10]:
from azureml.train.automl import AutoMLConfig
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "max_concurrent_iterations": 5,
                   "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task= 'classification',
    training_data= X,
    label_column_name= 'diagnosis',
    n_cross_validations= 5)

In [ ]:
! pip install none

In [ ]:
!pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)


ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Package name/Required version/Installed version
azureml-widgets/azureml-core~=1.20.0/azureml-core 1.21.0.post1
azureml-widgets/azureml-telemetry~=1.20.0/azureml-telemetry 1.21.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nPackage name/Required version/Installed version\nazureml-widgets/azureml-core~=1.20.0/azureml-core 1.21.0.post1\naz

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
best_run, fitted_model = remote_run.get_output()
print('Best-run model: ',best_run)
print('\n fitted model: ',fitted_model)

Best-run model:  Run(Experiment: udacity-capstone,
Id: AutoML_749f6e53-0471-4c90-be45-05db93f30f23_56,
Type: None,
Status: Completed)

 fitted model:  Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.01,
                                                                                                  min_weight_fraction_l

In [15]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_model,"best_model.pkl")

['best_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [17]:
script_file_name ='score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')
env=Environment.get(workspace=ws,name="AzureML-AutoML")

In [18]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=env)

In [20]:
from azureml.core.model import Model
import joblib
model = Model.register(workspace=ws, model_path="best_model.pkl", model_name="automl-test-model")

Registering model automl-test-model


In [21]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-Minimal")

In [22]:


aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'automl-cancer'
print(aci_service_name)

automl-cancer


In [23]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice



service_name = 'custom-env-service'



service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[automl_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [24]:

print("Scoring uri:" , service.scoring_uri)

Scoring uri: http://653b647c-cacf-4b6b-839e-86a815e7771e.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [25]:
scoring_uri = 'http://653b647c-cacf-4b6b-839e-86a815e7771e.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

input_data = json.dumps({'data':[{
    								
    'radius_mean':17.99,
    'texture_mean':10.38,
    'perimeter_mean':122.80,	
    'area_mean':1001.00,	
    'smoothness_mean':0.12,	
    'compactness_mean':0.28,
    'concavity_mean':0.30,	
    'concave points_mean':0.15	
    }
    ]
        })
response = requests.post(scoring_uri, data=input_data, headers=headers)

print("Result:",response.text)

Result: "{\"result\": [1]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [34]:
print(service.get_logs())

2021-01-30T14:26:42,218472904+00:00 - iot-server/run 
2021-01-30T14:26:42,220589920+00:00 - rsyslog/run 
2021-01-30T14:26:42,220615920+00:00 - gunicorn/run 
2021-01-30T14:26:42,221779129+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [51]:
service.delete()